In [1]:
import pandas as pd
import numpy as np

%pylab inline
import seaborn as sns
sns.set_style('darkgrid')

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


,Unnamed: 0,after_days,avg_sec_between_solutions,avg_solution_length,max_sec_between_solutions,max_solution_length,min_sec_between_solutions,min_solution_length,num_active_days,num_distinct_exercises,num_distinct_modules,num_empty_solutions,num_exercises,num_unsolved,std_sec_between_solutions,std_solution_length
0,0,161,1489728.07812,0.000000,58525736,0,10865,0,2,63,1,65,65,0,7370600.86877,0.000000
1,1,0,15656185.91670,7.640000,189786855,191,14078,0,4,25,25,24,25,0,41137923.16730,38.200000
2,2,0,7232903.08772,591.155172,403900681,1548,10072,0,2,58,1,5,58,0,53007775.47600,445.827470
3,3,0,-1.00000,0.000000,-1,0,-1,0,1,1,1,1,1,0,0.00000,NaN
4,4,0,1228668.28000,550.218905,134386926,2321,7874,0,4,201,3,11,201,0,10303660.59080,538.992098


In [3]:
df = pd.read_csv('user_data.csv')
del df['Unnamed: 0']
df['is_churned'] = (df['after_days'] == 0)
del df['after_days']
df.head()

,avg_sec_between_solutions,avg_solution_length,max_sec_between_solutions,max_solution_length,min_sec_between_solutions,min_solution_length,num_active_days,num_distinct_exercises,num_distinct_modules,num_empty_solutions,num_exercises,num_unsolved,std_sec_between_solutions,std_solution_length,is_churned
0,1489728.07812,0.000000,58525736,0,10865,0,2,63,1,65,65,0,7370600.86877,0.000000,False
1,15656185.91670,7.640000,189786855,191,14078,0,4,25,25,24,25,0,41137923.16730,38.200000,True
2,7232903.08772,591.155172,403900681,1548,10072,0,2,58,1,5,58,0,53007775.47600,445.827470,True
3,-1.00000,0.000000,-1,0,-1,0,1,1,1,1,1,0,0.00000,NaN,True
4,1228668.28000,550.218905,134386926,2321,7874,0,4,201,3,11,201,0,10303660.59080,538.992098,True


In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score

clf = GradientBoostingClassifier()

X = df[[c for c in df.columns if c != 'is_churned']]
y = df['is_churned']

X.fillna('-1', inplace=True)
       
scores = []
for train_index, test_index in KFold(n=len(X), n_folds=5, shuffle=True, random_state=2016):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict_proba(X_test)[:,1]

    score = roc_auc_score(y_test, predictions)
    scores.append(score)
        
scores = np.array(scores)
print scores

[ 0.82727676  0.82665974  0.82517307  0.82319762  0.82517257]


/usr/local/lib/python2.7/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
